In [20]:
import datetime
import sys
import os

import IPython
import IPython.display
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from cond_rnn import ConditionalRNN
from sklearn.preprocessing import MinMaxScaler
from pmdarima.arima import auto_arima

import src.preprocessing
from src.functions import load_data, TimeSeriesTensor, create_evaluation_df, series_to_supervised, plot_train_history, validation

np.set_printoptions(threshold=sys.maxsize)

In [16]:
metrics = pd.DataFrame(columns=['mape','rmse'], index=range(1,11))
#for i in range(1,28):
i = 1
filename = '../data/Columbia_clean/Residential_'+str(i)+'.csv'
df = pd.read_csv(filename, index_col=0)
train_inputs, test_inputs, train_c1, test_c1, train_c2, test_c2 = ConditionalRNN_data_preparation(df)

ValueError: too many values to unpack (expected 6)

In [21]:
class Baseline(tf.keras.Model):
    def __init__(self, label_index=None):
        super().__init__()
        self.label_index = label_index

    def call(self, inputs):
        if self.label_index is None:
            return inputs
        result = inputs[:, :, self.label_index]
        return result[:, :, tf.newaxis]

In [22]:
metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(27))
from progressbar import ProgressBar
pbar = ProgressBar()
for i in pbar(range(1,28)):
    filename = '../data/Columbia_clean/Residential_'+str(i)+'.csv'
    df = pd.read_csv(filename, index_col=0)
    train_inputs, test_inputs, y_scaler = MIMO_fulldata_preparation(df)
    baseline = Baseline(label_index = 0)
    baseline.compile(loss=tf.losses.MeanSquaredError(), metrics=[tf.metrics.MeanSquaredError()])
    predictions_B = baseline.predict(test_inputs['X'])
    eval_df_B = create_evaluation_df(predictions_B.reshape(-1,24), test_inputs, 24, y_scaler)
    mape = validation(eval_df_B['prediction'], eval_df_B['actual'], 'MAE')
    rmse = validation(eval_df_B['prediction'], eval_df_B['actual'], 'RMSE')
    metrics.loc[i-1] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': i})
#metrics.to_csv('./results/Columbia/baseline.csv')

100% |########################################################################|


In [ ]:
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24

FULL_LSTMIMO = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, input_shape=(24, 15)),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(HORIZON)
])

metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(27))
from progressbar import ProgressBar
pbar = ProgressBar()
for i in pbar(range(0,27)):
    filename = '../data/Columbia_clean/Residential_'+str(i+1)+'.csv'
    df = pd.read_csv(filename, index_col=0)
    train_longer_inputs, test_longer_inputs, y_scaler = MIMO_fulldata_preparation(df)
    # full data LSTM MIMO compilation and fit
    FULL_LSTMIMO.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    for j in range(0,10):
        print('RUN:{}'.format(j))
        history = FULL_LSTMIMO.fit(train_longer_inputs['X'], train_longer_inputs['target'], batch_size=32, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)

        FD_predictions = FULL_LSTMIMO.predict(test_longer_inputs['X'])
        FD_eval_df = create_evaluation_df(FD_predictions, test_longer_inputs, HORIZON, y_scaler)
        mae = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAE')
        mape = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAPE')
        rmse = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'RMSE')
        #print('rmse {}'.format(rmse))
        metrics.loc[10*i + j] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': i})
metrics.to_csv('./results/Columbia/10runs/2fulldata_LSTM.csv')

RUN:0
Epoch 1/100
656/656 [==============================] - 12s 18ms/step - loss: 0.0036 - mean_squared_error: 0.0036 - val_loss: 0.0037 - val_mean_squared_error: 0.0037
Epoch 2/100
656/656 [==============================] - 10s 15ms/step - loss: 0.0026 - mean_squared_error: 0.0026 - val_loss: 0.0034 - val_mean_squared_error: 0.0034
Epoch 3/100
656/656 [==============================] - 10s 15ms/step - loss: 0.0025 - mean_squared_error: 0.0025 - val_loss: 0.0032 - val_mean_squared_error: 0.0032
Epoch 4/100
656/656 [==============================] - 10s 16ms/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0030 - val_mean_squared_error: 0.0030
Epoch 5/100
656/656 [==============================] - 10s 16ms/step - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.0029 - val_mean_squared_error: 0.0029
Epoch 6/100
656/656 [==============================] - 9s 14ms/step - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.0029 - val_mean_squared_error: 0.0029
Epoch

656/656 [==============================] - 9s 14ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0027 - val_mean_squared_error: 0.0027
Epoch 11/100
656/656 [==============================] - 10s 15ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0027 - val_mean_squared_error: 0.0027
Epoch 12/100
656/656 [==============================] - 9s 14ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0028 - val_mean_squared_error: 0.0028
Epoch 13/100
656/656 [==============================] - 9s 14ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0027 - val_mean_squared_error: 0.0027
Epoch 14/100
656/656 [==============================] - 9s 14ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0028 - val_mean_squared_error: 0.0028
Epoch 15/100
656/656 [==============================] - 9s 14ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0029 - val_mean_squared_error: 0.0029
Epoch 16/100
656/656 [

656/656 [==============================] - 14s 22ms/step - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.0030 - val_mean_squared_error: 0.0030
RUN:6
Epoch 1/100
656/656 [==============================] - 15s 24ms/step - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.0029 - val_mean_squared_error: 0.0029
Epoch 2/100
656/656 [==============================] - 15s 22ms/step - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.0030 - val_mean_squared_error: 0.0030
Epoch 3/100
656/656 [==============================] - 14s 21ms/step - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.0030 - val_mean_squared_error: 0.0030
Epoch 4/100
656/656 [==============================] - 15s 23ms/step - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.0030 - val_mean_squared_error: 0.0030
Epoch 5/100
656/656 [==============================] - 14s 22ms/step - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.0031 - val_mean_squared_error: 0.0031
Epoch 6/100
656/

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



682/682 [==============================] - 9s 14ms/step - loss: 0.0054 - mean_squared_error: 0.0054 - val_loss: 0.0067 - val_mean_squared_error: 0.0067
Epoch 9/100
682/682 [==============================] - 9s 14ms/step - loss: 0.0054 - mean_squared_error: 0.0054 - val_loss: 0.0069 - val_mean_squared_error: 0.0069
Epoch 10/100
682/682 [==============================] - 9s 13ms/step - loss: 0.0053 - mean_squared_error: 0.0053 - val_loss: 0.0069 - val_mean_squared_error: 0.0069
Epoch 11/100
682/682 [==============================] - 9s 13ms/step - loss: 0.0053 - mean_squared_error: 0.0053 - val_loss: 0.0069 - val_mean_squared_error: 0.0069
Epoch 12/100
682/682 [==============================] - 9s 13ms/step - loss: 0.0052 - mean_squared_error: 0.0052 - val_loss: 0.0070 - val_mean_squared_error: 0.0070
RUN:1
Epoch 1/100
682/682 [==============================] - 9s 13ms/step - loss: 0.0052 - mean_squared_error: 0.0052 - val_loss: 0.0071 - val_mean_squared_error: 0.0071
Epoch 2/100
682/682

...

RUN:0
Epoch 1/100
577/577 [==============================] - 11s 19ms/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.0065 - val_mean_squared_error: 0.0065
Epoch 2/100
577/577 [==============================] - 10s 18ms/step - loss: 0.0074 - mean_squared_error: 0.0074 - val_loss: 0.0065 - val_mean_squared_error: 0.0065
Epoch 3/100
577/577 [==============================] - 10s 18ms/step - loss: 0.0071 - mean_squared_error: 0.0071 - val_loss: 0.0065 - val_mean_squared_error: 0.0065
Epoch 4/100
577/577 [==============================] - 10s 18ms/step - loss: 0.0069 - mean_squared_error: 0.0069 - val_loss: 0.0065 - val_mean_squared_error: 0.0065
Epoch 5/100
577/577 [==============================] - 10s 17ms/step - loss: 0.0068 - mean_squared_error: 0.0068 - val_loss: 0.0065 - val_mean_squared_error: 0.0065
Epoch 6/100
577/577 [==============================] - 10s 17ms/step - loss: 0.0067 - mean_squared_error: 0.0067 - val_loss: 0.0066 - val_mean_squared_error: 0.0066
Epoc

577/577 [==============================] - 9s 16ms/step - loss: 0.0056 - mean_squared_error: 0.0056 - val_loss: 0.0070 - val_mean_squared_error: 0.0070
Epoch 15/100
577/577 [==============================] - 9s 16ms/step - loss: 0.0056 - mean_squared_error: 0.0056 - val_loss: 0.0070 - val_mean_squared_error: 0.0070
Epoch 16/100
577/577 [==============================] - 9s 16ms/step - loss: 0.0056 - mean_squared_error: 0.0056 - val_loss: 0.0070 - val_mean_squared_error: 0.0070
Epoch 17/100
577/577 [==============================] - 9s 16ms/step - loss: 0.0056 - mean_squared_error: 0.0056 - val_loss: 0.0072 - val_mean_squared_error: 0.0072
Epoch 18/100
577/577 [==============================] - 9s 16ms/step - loss: 0.0056 - mean_squared_error: 0.0056 - val_loss: 0.0071 - val_mean_squared_error: 0.0071
Epoch 19/100
577/577 [==============================] - 9s 16ms/step - loss: 0.0056 - mean_squared_error: 0.0056 - val_loss: 0.0072 - val_mean_squared_error: 0.0072
Epoch 20/100
577/577 [=

Epoch 8/100
577/577 [==============================] - 9s 15ms/step - loss: 0.0054 - mean_squared_error: 0.0054 - val_loss: 0.0074 - val_mean_squared_error: 0.0074
Epoch 9/100
577/577 [==============================] - 9s 15ms/step - loss: 0.0054 - mean_squared_error: 0.0054 - val_loss: 0.0073 - val_mean_squared_error: 0.0073
Epoch 10/100
577/577 [==============================] - 9s 15ms/step - loss: 0.0054 - mean_squared_error: 0.0054 - val_loss: 0.0074 - val_mean_squared_error: 0.0074
Epoch 11/100
577/577 [==============================] - 9s 15ms/step - loss: 0.0054 - mean_squared_error: 0.0054 - val_loss: 0.0074 - val_mean_squared_error: 0.0074
RUN:7
Epoch 1/100
577/577 [==============================] - 8s 14ms/step - loss: 0.0054 - mean_squared_error: 0.0054 - val_loss: 0.0073 - val_mean_squared_error: 0.0073
Epoch 2/100
577/577 [==============================] - 8s 15ms/step - loss: 0.0054 - mean_squared_error: 0.0054 - val_loss: 0.0074 - val_mean_squared_error: 0.0074
Epoch 3/

Epoch 24/100
577/577 [==============================] - 9s 16ms/step - loss: 0.0053 - mean_squared_error: 0.0053 - val_loss: 0.0075 - val_mean_squared_error: 0.0075
Epoch 25/100
577/577 [==============================] - 9s 16ms/step - loss: 0.0053 - mean_squared_error: 0.0053 - val_loss: 0.0075 - val_mean_squared_error: 0.0075
Epoch 26/100
577/577 [==============================] - 10s 17ms/step - loss: 0.0053 - mean_squared_error: 0.0053 - val_loss: 0.0075 - val_mean_squared_error: 0.0075
Epoch 27/100
577/577 [==============================] - 10s 17ms/step - loss: 0.0053 - mean_squared_error: 0.0053 - val_loss: 0.0074 - val_mean_squared_error: 0.0074
Epoch 28/100
577/577 [==============================] - 10s 18ms/step - loss: 0.0053 - mean_squared_error: 0.0053 - val_loss: 0.0075 - val_mean_squared_error: 0.0075
Epoch 29/100
577/577 [==============================] - 10s 17ms/step - loss: 0.0053 - mean_squared_error: 0.0053 - val_loss: 0.0075 - val_mean_squared_error: 0.0075


RUN:0
Epoch 1/100
577/577 [==============================] - 11s 19ms/step - loss: 0.0051 - mean_squared_error: 0.0051 - val_loss: 0.0042 - val_mean_squared_error: 0.0042
Epoch 2/100
577/577 [==============================] - 10s 18ms/step - loss: 0.0046 - mean_squared_error: 0.0046 - val_loss: 0.0042 - val_mean_squared_error: 0.0042
Epoch 3/100
577/577 [==============================] - 11s 19ms/step - loss: 0.0045 - mean_squared_error: 0.0045 - val_loss: 0.0042 - val_mean_squared_error: 0.0042
Epoch 4/100
577/577 [==============================] - 10s 17ms/step - loss: 0.0043 - mean_squared_error: 0.0043 - val_loss: 0.0042 - val_mean_squared_error: 0.0042
Epoch 5/100
577/577 [==============================] - 10s 17ms/step - loss: 0.0042 - mean_squared_error: 0.0042 - val_loss: 0.0042 - val_mean_squared_error: 0.0042
Epoch 6/100
577/577 [==============================] - 10s 17ms/step - loss: 0.0041 - mean_squared_error: 0.0041 - val_loss: 0.0043 - val_mean_squared_error: 0.0043
Epoc

In [ ]:
NUM_CELLS = 32
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24
ConditionalRNN = Sequential(layers=[ConditionalRNN(32, cell='LSTM'),
                                    Dense(HORIZON)])

metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(11))
from progressbar import ProgressBar
pbar = ProgressBar()
for i in pbar(range(0,27)):
    filename = '../data/Columbia_clean/Residential_'+str(i+1)+'.csv'
    df = pd.read_csv(filename, index_col=0)
    train_inputs, test_inputs, train_c1, test_c1, train_c2, test_c2, y_scaler = ConditionalRNN_data_preparation(df)
    # full data LSTM MIMO compilation and fit
    ConditionalRNN.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    for j in range(0,10):
        history = ConditionalRNN.fit([train_inputs['X'],train_c1,train_c2], train_inputs['target'], batch_size=32, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)

        #save_model(ConditionalRNN, i)
        predictions = ConditionalRNN.predict([test_inputs['X'],test_c1,test_c2])
        eval_df = create_evaluation_df(predictions, test_inputs, HORIZON, y_scaler)
        mae = validation(eval_df['prediction'], eval_df['actual'], 'MAE')
        rmse = validation(eval_df['prediction'], eval_df['actual'], 'RMSE')
        mape = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAPE')
        # add results for a run j and a building i
        metrics.loc[10*i + j] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': i})
metrics.to_csv('./results/Columbia/10runs/2ConditionalRNN10_sequential.csv')

In [ ]:
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24

LSTMIMO = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, input_shape=(24, 5)),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(HORIZON)
])

metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(11))
from progressbar import ProgressBar
pbar = ProgressBar()
for i in pbar(range(0,27)):
    filename = '../data/Columbia_clean/Residential_'+str(i+1)+'.csv'
    df = pd.read_csv(filename, index_col=0)
    train_inputs, test_inputs, _, _, _, _, y_scaler = ConditionalRNN_data_preparation(df)
    # full data LSTM MIMO compilation and fit
    LSTMIMO.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    
    for j in range(0,10):
        history = LSTMIMO.fit(train_inputs['X'], train_inputs['target'], batch_size=32, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)

        FD_predictions = LSTMIMO.predict(test_inputs['X'])
        FD_eval_df = create_evaluation_df(FD_predictions, test_inputs, HORIZON, y_scaler)
        mae = validation(eval_df['prediction'], eval_df['actual'], 'MAE')
        rmse = validation(eval_df['prediction'], eval_df['actual'], 'RMSE')
        mape = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAPE')
        #print('rmse {}'.format(rmse))
        metrics.loc[10*i + j] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': i})
metrics.to_csv('./results/Columbia/10runs/2LSTM_MIMO10.csv')

In [17]:
def train_test_split(df, n_test):
    test_df = df.copy()[-(n_test+23):]
    train_df = df.copy()[:-(len(test_df)-23)]
    return train_df, test_df

In [18]:
def ConditionalRNN_data_preparation(df, n_test=4380, T=24, HORIZON=24):
    df = src.preprocessing.preprocess(df, 'Canada')
    c2 = series_to_supervised(df)
    c1 = df.iloc[:,1:9]
    c1 = c1.loc[c2.index]
    short_df = df.iloc[:,[0,1,-4,-3,-2,-1]].copy()
    #check how to formulate more correctly
    short_df = short_df[146:]
    train_df, test_df = train_test_split(short_df, n_test)
    train_c1, test_c1 = train_test_split(c1, n_test)
    train_c2, test_c2 = train_test_split(c2, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    train_c2[['value(t-24)']] = y_scaler.fit_transform(train_c2[['value(t-24)']])
    train_c2[['value(t-168)']] = y_scaler.fit_transform(train_c2[['value(t-168)']])
    X_scaler = MinMaxScaler()
    train_df[train_df.columns] = X_scaler.fit_transform(train_df)
    test_df[train_df.columns] = X_scaler.fit_transform(test_df)
    tensor_structure = {'X':(range(-T+1, 1), ['value','fractional hour_sin','fractional hour_cos','day of year_sin','day of year_cos'])}
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    train_c1 = c1.reindex(train_inputs.dataframe.index)
    train_c2 = c2.reindex(train_inputs.dataframe.index)
    test_c1 = c1.reindex(test_inputs.dataframe.index)
    test_c2 = c2.reindex(test_inputs.dataframe.index)
    return train_inputs, test_inputs, train_c1, test_c1, train_c2, test_c2, y_scaler

In [19]:
def MIMO_fulldata_preparation(df, n_test=4380, T=24, HORIZON=24):
    df = src.preprocessing.preprocess(df, 'Canada')
    df = df.merge(series_to_supervised(df), how='right', left_index=True, right_index=True)
    train_df, test_df = train_test_split(df, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    long_scaler = MinMaxScaler()
    test_df[test_df.columns] = long_scaler.fit_transform(test_df)
    train_df[train_df.columns] = long_scaler.fit_transform(train_df)
    tensor_structure = {'X':(range(-T+1, 1), train_df.columns)}
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    return train_inputs, test_inputs, y_scaler